In [ ]:
### Install very uncommon dependencies that are necessary to call the APIs
# Google Sheets and Google Drive APIs
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install gspread oauth2client

### Hello Sign APIs
!pip install hellosign-python-sdk

In [ ]:
### Imports libraries and methods to initialize Google Drive API service
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import io
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload

### Imports Libraries and methods to Initialize Google Sheets API Service
import gspread
from oauth2client.service_account import ServiceAccountCredentials

### Imports hellosign SDK and Initializes Client from HelloSign API
from hellosign_sdk import HSClient
import datetime
import time

In [ ]:
hello_sign_api_key = "" # Insert Hello Sign API Key
sheet_file_id = "" # Sheet File ID
sheet_creds_json_keyfile = "" # Path to credentials for GSheets API. Download from GCP
drive_client_secret_file = "" # Path to credentials for GDrive API. Download from GCP
drive_token_file_name = "token.pickle"

folder_id = "" # GDrive Folder ID to upload signed offer letters

# Paths on your local machine for draft and signed offer letters
drafted_path = 'DraftedOfferLettersByScript/#' # Adds # to identify those are drafted letters
completed_path = 'SignedOfferLettersByScript/'

# Code of action per column in the Google Sheet
actions_by_column = {
  2: "StudentEmail",
  3: "StudentFullName",
  4: "Citizenship",
  6: "Eligibility",
  7: "Rotation",
  8: "Company",
  9: "Role",
  10: "Manager",
  11: "Salary",
  12: "Project Code",
  13: "CTD Sign Off",
  14: "PaycomID",
  17: "Dept Code",
  18: "Manager Email",
  101: "Doc_Unsigned_Offer_Link",
  102: "PDF_Unsigned_Offer_Link",
  103: "HelloSign_Offer_UUID",
  104: "Signed_Offer_Link",
  105: "Onboarding_Email_Sent"
}

In [ ]:
### Google Sheets API functions

# Posts update in updates sheet
# Inspired from postUpdatedStudentInfoAsUpdate function in the Apps Script implementation

def post_updated_info_as_update(updates_sheet,
                                last_row_index,
                                today, 
                                email, 
                                name, 
                                paycomID, 
                                action_num, 
                                value):
    if value:
        action_code = actions_by_column[action_num]
        
        updates_sheet.update_cell(last_row_index, 1, today)
        updates_sheet.update_cell(last_row_index, 2, email)
        updates_sheet.update_cell(last_row_index, 3, name)
        updates_sheet.update_cell(last_row_index, 4, paycomID)
        updates_sheet.update_cell(last_row_index, 5, action_code)
        updates_sheet.update_cell(last_row_index, 6, value)

In [ ]:
### Google Drive API functions

# Downloads the file using the Google Drive API
def download_file(drive_service, file_id, drafted_path):
    # HTTP Request to download a Google Doc file as pdf format
    request = drive_service.files().export(fileId = file_id, mimeType='application/pdf')
    # HTTP GET Request to obtain the metadata of the downloaded file
    filename = drive_service.files().get(fileId = file_id, supportsAllDrives=True).execute()["name"]
    filename = filename.replace("/", "-")
    filename = filename.replace("Design/Researcher", "Design-Researcher")
    
    # Saves the file in our local machine to send for signing
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print("Download %d%%." % int(status.progress() * 100), filename)
    with io.open(drafted_path + filename + '.pdf', 'wb') as f:
        fh.seek(0)
        f.write(fh.read())
        
    return filename

        
        
def upload_file(drive_service, file_name, folder_id, completed_path):
    file_metadata = {'name': file_name + '.pdf'}
    media = MediaFileUpload(completed_path + file_name + '.pdf', 
                            mimetype = 'application/pdf')
    
    file = drive_service.files().create(body = file_metadata,
                                       media_body = media,
                                       fields = 'id, parents').execute()

    # Move the file to the new folder
    moved_file = drive_service.files().update(fileId = file.get('id'),
                                              supportsAllDrives=True,
                                              addParents = folder_id,
                                              removeParents = file.get('parents')[0],
                                              fields='id, parents').execute()

    print('Uploaded Letter for ', file_name)
    
    return moved_file.get('id')

In [ ]:
### Hello Sign API Functions

def send_sign_req(sign_client, filename, name, email, drafted_path):
    # Sends the Document through Hello Sign
    sign_req = sign_client.send_signature_request(
        test_mode = False,
        title = filename + ' WS AY 21-22 Offer Letter',
        subject = filename + ' WS AY 21-22 Offer Letter',
        message = 'Hi, Please review and sign the offer letter through Hello Sign as part of your Work Study position at Minerva. Best!',
        signers = [
            {'email_address': email, 'name': name},
            # CFO Email is a must in each letter. Inputs as fixed string
            # {'email_address': 'cfo@minerva.edu', 'name': 'CFOs Name'}
        ],
#         cc_email_addresses=[
#             'asantacruz@uni.minerva.edu', 
#         ],
        use_text_tags = True,
        hide_text_tags = True,
        files = [drafted_path + filename + '.pdf'])

    print('Used HelloSign to send Letter to ', name, email)
    
    return sign_req


def download_completed_offer(sign_client, sign_req, file_title, sign_req_id, completed_path):
    sign_client.get_signature_request_file(signature_request_id = sign_req_id,
                                                        filename = completed_path + file_title + '.pdf',
                                                        file_type ='pdf')

    
    
def send_reminder(sign_client, sign_req_id, hire_email):
    sign_client.remind_signature_request(signature_request_id = sign_req_id, email_address = hire_email)
    


In [ ]:
### Main Function

def main(updates_sheet, checklist_sheet, drive_service, sign_client, folder_id, drafted_path, completed_path):
    """
    Input: sheet (Obj). Google Sheets Client File Object. Supposed to open the offers Tracker Sheet
            drive_service (Obj). Google Drive Client. Bridges the Script with an authorized Project with Google Drive API enabled
            sign_client (Obj). Hello Sign Client to make the documents completion verification and send signature requests
            folder_id (String). Google Drive Folder ID to upload the completed files
            draft_path (String): Path in local machine to store drafted documents templates from Google Drive
            completed_path (String): Path in local machine to store the completed documents from Hello Sign
    
    Ouput: Depending on the conditions and status on the HR Spreadsheet
    - Downloads drafted documents to be signed from Googe Drive to our local machine on a specified path
    - Sends signature requests with document template through Hello Sign
    - Downloads completed documents from Hello SIgn to local machine
    - Uploads completed file to a specific folder in Google Drive
    """
    hires = checklist_sheet.get_all_values()
    num_rows_updates_sheet = len(updates_sheet.get_all_values())
    # day_of_week = datetime.date.today().strftime('%A')
    today = datetime.date.today().strftime("%m/%d/%Y")
    
    for i in range(1, len(hires)):
        hire_name = hires[i][0]
        hire_email = hires[i][1]
        paycomID = hires[i][8]
        
        # Brings the DocID number for the download
        file_url = hires[i][14]
        hellosign_offer_uuid = hires[i][16]
        signed_offer_url = hires[i][17]
        
        if file_url and not hellosign_offer_uuid:
            file_id = str(file_url.split("/")[5])
            
            print(hire_name, hire_email, paycomID)
            
            # Downloads the file using the Google Drive API
            file_name = download_file(drive_service, file_id, drafted_path)
                            
            # Sends the Document through Hello Sign
            sign_req = send_sign_req(sign_client, file_name, hire_name, hire_email, drafted_path)
            
            # Updates Status of the Offer Letter in the Google Sheet
            post_updated_info_as_update(updates_sheet,
                                num_rows_updates_sheet + 1,
                                today, 
                                hire_email, 
                                hire_name, 
                                paycomID, 
                                103, 
                                sign_req.signature_request_id)
            num_rows_updates_sheet += 1
            
            print('Updated Offer Letter status for', hire_name, hire_email)
            
            time.sleep(7)
            
            print("")
            print("---------------------------------------------------------------------------------")
            print("")
            
        elif file_url and hellosign_offer_uuid and not signed_offer_url:
            sign_req = sign_client.get_signature_request(hellosign_offer_uuid)
            sign_req_created_datetime = datetime.date.fromtimestamp(sign_req.created_at)
            file_title = sign_req.title
            if sign_req.is_complete == True:
                # Downloads Signed Offer Letter to local machine
                download_completed_offer(sign_client, 
                                         sign_req, 
                                         file_title, 
                                         hellosign_offer_uuid, 
                                         completed_path)

                # Uploads Offer Letter to Google Drive using the API (to folder with folder_id)
                signed_file_id = upload_file(drive_service, file_title, folder_id, completed_path)

                signed_file_url = f"https://drive.google.com/file/d/{signed_file_id}/view"
                
                post_updated_info_as_update(updates_sheet,
                                num_rows_updates_sheet + 1,
                                today, 
                                hire_email, 
                                hire_name, 
                                paycomID, 
                                104, 
                                signed_file_url)
                num_rows_updates_sheet += 1
                
                print('Updated Offer Letter status for ', file_title)
                
                time.sleep(7)
                
                print("")
                print("---------------------------------------------------------------------------------")
                print("")
            
            elif abs((sign_req_created_datetime - datetime.date.today()).days) >= 1:
                hire_email = hires[i][1]
                
                # print(sign_req_created_datetime)
                # Sends reminder to hires whose signatures are pending
                send_reminder(sign_client, hellosign_offer_uuid, hire_email)

                print('Sent reminder for ', file_title)

                print("")
                print("---------------------------------------------------------------------------------")
                print("")


In [ ]:
def agol(hello_sign_api_key, 
         sheet_file_id, 
         sheet_creds_json_keyfile, 
         drive_client_secret_file, 
         drive_token_file_name, 
         folder_id, 
         drafted_path, 
         completed_path):
    
    # Initializes a client using my corporate email API Key
    sign_client = HSClient(api_key=hello_sign_api_key)

    print("Defined Hello Sign Client")

    # Initializes Google Sheets API Service from client
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(sheet_creds_json_keyfile, scope)
    client = gspread.authorize(creds)

    # Obtains a Google Sheets service
    sheet = client.open_by_key(sheet_file_id)
    updates_sheet = sheet.worksheet("Updates")
    checklist_sheet = sheet.worksheet("Offers Checklist")

    print("Defined Google Sheets Service")


    # Initializes Google Drive API Service from client
    SCOPES = ['https://www.googleapis.com/auth/drive']
    creds = None

    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time.
    if os.path.exists(drive_token_file_name):
        with open(drive_token_file_name, 'rb') as token:
            creds = pickle.load(token)

    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                drive_client_secret_file, SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the credentials for the next run
        with open(drive_token_file_name, 'wb') as token:
            pickle.dump(creds, token)

    # Obtains a Google Drive service
    drive_service = build('drive', 'v3', credentials=creds)

    print("Defined Google Drive Service")

    main(updates_sheet, checklist_sheet, drive_service, sign_client, folder_id, drafted_path, completed_path)

In [ ]:
agol(hello_sign_api_key, 
         sheet_file_id, 
         sheet_creds_json_keyfile, 
         drive_client_secret_file, 
         drive_token_file_name, 
         folder_id, 
         drafted_path, 
         completed_path)